Data Communication & Networking - Server

In [ ]:
import socket
import threading

HOST = '0.0.0.0'
PORT = 12345
LOG_FILE = 'chat_history.txt'

clients = []
username = {}

def broadcast(message, exclude=None):
    for client in clients:
        if client != exclude:
            try:
                client.sendall(message.encode())
            except:
                pass
            
def log_message(message):
    with open(LOG_FILE, 'a', encoding='utf-8') as f:
        f.write(message + '\n')
        
def handle_client(client, addr):
    global username
    try:
        client.sendall("Enter username - ".encode())
        user = client.recv(1024).decode().strip()
        username[client] = user
        join_message = f"{user} has joined the chat."
        broadcast(join_message)
        log_message(join_message)
        print(join_message)

        while True:
            message = client.recv(1024)
            if not message:
                break
            text = message.decode().strip()
            full_message = f"{user} - {text}"
            broadcast(full_message, exclude=client)
            log_message(full_message)
            print(full_message)

    except:
        pass
    finally:
        leave_message = f"{username.get(client, 'Someone')} has left the chat."
        broadcast(leave_message, exclude=client)
        log_message(leave_message)
        print(leave_message)
        clients.remove(client)
        client.close()
        username.pop(client, None)
        
def main():
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind((HOST, PORT))
    server.listen()
    print(f"server is listening on {HOST}:{PORT}")
    while True:
        client, addr = server.accept()
        clients.append(client)
        threading.Thread(target=handle_client, args=(client, addr), daemon=True).start()
        
if __name__ == "__main__":
    main()